In [1]:
from keras import layers
from keras import models
import numpy as np
import os
from PIL import Image
import shutil
from flask import Flask

Using TensorFlow backend.


In [2]:
train_dir = os.path.join('/home/wenceslai/Documents/keggle_cats_dogs', 'train_dir')
validation_dir = os.path.join('/home/wenceslai/Documents/keggle_cats_dogs', 'validation_dir')

dog_img_dir = '/home/wenceslai/Documents/keggle_cats_dogs/PetImages/Dog'
cat_img_dir = '/home/wenceslai/Documents/keggle_cats_dogs/PetImages/Cat'

train_dog_dir = os.path.join(train_dir, 'train_dog_dir')
val_dog_dir = os.path.join(validation_dir, 'val_dog_dir')

train_cat_dir = os.path.join(train_dir, 'train_cat_dir')
val_cat_dir = os.path.join(validation_dir, 'val_cat_dir')

In [ ]:
os.mkdir(validation_dir)
os.mkdir(train_dir)

os.mkdir(train_dog_dir)
os.mkdir(val_dog_dir)

os.mkdir(train_cat_dir)
os.mkdir(val_cat_dir)

In [25]:
TRAIN_SET_LEN = 2000 
VAL_SET_LEN = 500 
 
for i in range(TRAIN_SET_LEN,TRAIN_SET_LEN + VAL_SET_LEN): 
    try: 
        img = Image.open(os.path.join(dog_img_dir, '{}.jpg'.format(i))).convert('RGB')
      
    except(IOError, SyntaxError) as e: 
        print("error image: dog.{}.jpg couldnt be opened".format(i)) 
        continue
        
    if i < TRAIN_SET_LEN: 
        img.save(os.path.join(train_dog_dir,'dog.{}.jpg'.format(i))) 
         
    elif TRAIN_SET_LEN < i < TRAIN_SET_LEN + VAL_SET_LEN: 
        img.save(os.path.join(val_dog_dir,'dog.{}.jpg'.format(i)))   
        
print("doggos done") 

doggos done


In [21]:
for i in range(TRAIN_SET_LEN + VAL_SET_LEN): 
    try: 
        img = Image.open(os.path.join(cat_img_dir, '{}.jpg'.format(i))).convert('RGB') 
        
    except(IOError, SyntaxError) as e: 
        print("error image: cat.{}.jpg couldt be opened".format(i)) 
        continue
        
    if i < TRAIN_SET_LEN: 
        img.save(os.path.join(train_cat_dir, 'cat.{}.jpg'.format(i))) 
         
    elif TRAIN_SET_LEN < i < TRAIN_SET_LEN + VAL_SET_LEN: 
        img.save(os.path.join(val_cat_dir, 'cat.{}.jpg'.format(i))) 
 
print("kitties done")           

error image: cat.666.jpg couldt be opened
kitties done


In [3]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(200, 200, 3))

from keras import activations

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
conv_base.trainable = False
print("nuber of trainable weight after freezing: ", len(model.trainable_weights))

nuber of trainable weight after freezing:  4


In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200,200),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(200,200),
    batch_size=20,
    class_mode='binary'
)
from keras import optimizers, losses

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), 
              metrics=['acc'], 
              loss='binary_crossentropy')

Found 3999 images belonging to 2 classes.
Found 998 images belonging to 2 classes.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=1
)



Epoch 1/15
100/100 [==============================] - 352s 4s/step - loss: 0.5633 - acc: 0.7200 - val_loss: 0.3953 - val_acc: 0.8327

In [2]:
history_dict = history


NameError: name 'history' is not defined

In [7]:
#Save the model
# serialize model to JSON
model_json = model.to_json()
with open("model_cats_dogs_VGG16.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_cats_dogs_VGG16.h5")
print("Saved model to disk")

Saved model to disk
